# Setup


Montando o disco do Google Drive

In [1]:
clone_repository()
update_repository()
install_packages()
  
#print('\nfiles:')
#!ls -a 
 
# rm unet -r
# rm /content/hedychium_coronarium/test/results/ -r

NameError: name 'clone_repository' is not defined

In [ ]:
while True:pass

'''
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

let seconds = 60
let intervalConnectButton = setInterval(ConnectButton, 60 * 1000);
'''

In [ ]:
def clone_repository():
  
  cur_dir = !pwd
  
  if (cur_dir[0] != '/content'):
    %cd '/content'

  if (cur_dir[0] == '/unet'):
    %cd ..

  !git clone https://github.com/charlespcamargo/unet.git unet
  print('\n')

def update_repository():
  % cd unet 
  ! git pull 
  % cd .. 

def install_packages():
  % cd unet 
  ! pip install -r requirements.txt  
  % cd .. 

In [ ]:
# analisando a quantidade de arquivos
# ! ls data/train/image/ -la | wc -l
# ! ls data/train/label/ -la | wc -l

# ! diff <(ls -a data/train/image) <(ls -a data/train/label/)

Verificando a versão do CUDA

In [ ]:
!nvcc --version
#!torch.cuda.is_available()
!nvidia-smi
!nvidia-smi -L
!lscpu |grep 'Model name'
!lscpu | grep 'Socket(s):'
!lscpu | grep 'Core(s) per socket:'
!lscpu | grep 'Thread(s) per core'
!lscpu | grep "L3 cache" 
!lscpu | grep "MHz"
!free -h --si | awk  '/Mem:/{print $2}'
!df -h / | awk '{print $4}'
!nvidia-settings -q CUDACores -t

Aumentando a memoria RAM


In [ ]:
# a = []
# while(1):
#     a.append('1')

Verificar a quantidade de memória RAM disponível

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


Configurando a versão do tensorflow

In [ ]:
import numpy as np
from tensorflow import keras
print(keras.__version__)

### pip install tensorflow
#% tensorflow_version 2.x

2.4.0


# Clone Database

Baixar o dataset

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='19m8djdqbBqofLmhydDWGSQ-xyvtmHzcm',
                                    dest_path='./hedychium_coronarium.zip',
                                    unzip=True)

Unzipping...Done.


# Execution

In [ ]:
update_repository()

/content/unet
Already up to date.
/content


In [ ]:
% cd unet
 
import easydict
from unet_helper import UnetHelper 
 
args = easydict.EasyDict({
    "t": 1, # "Informe o tipo  '--t -1' parametros, '--t 0' treino, '--t 1' teste, '--t 2' sumario', '--t 3' avaliacao, '--t 4' f-beta-score")
    "g": 0, # Gerar arquivos '--g 0' para nao gerar arquivos ou '--g 1' para gerar
    "q": 0, # Quantidade de arquivos para teste '--q 0' para nao gerar arquivos ou '--q 1' para gerar
    "n": None, # Informe o nome do arquivo de pesos para executar o teste ou ler o sumario
    "b": None, # Informe o beta para calcular o f-beta score
    "p": None, # Informe o precision para calcular o f-beta score
    "r": None # Informe o recall para calcular o f-beta score
})
 
helper = UnetHelper()

/content/unet


In [ ]:
helper = UnetHelper()
helper.set_arguments(epochs=15, steps_per_epoch=100, batch_size=4, base_folder='../hedychium_coronarium/')
helper.show_arguments()

## if you don't want to do data augmentation, set data_gen_args as an empty dict

In [ ]:
helper.generate_my_gen(args)

<generator object trainGenerator at 0x7efc34d39b50>

##Visualize your data augmentation result
you will see 60 transformed images and their masks in data/membrane/train/aug


In [ ]:
#you will see 60 transformed images and their masks in data/membrane/train/aug
num_batch = 3
for i,batch in enumerate(helper.my_gene):    
    if(i >= num_batch):
        break

Found 244 images belonging to 1 classes.
Found 244 images belonging to 1 classes.


#Executar o treinamento

##Treinar com as imagens

In [19]:
# % rm .logs/20210426/ -r
 
for i in range(0, 20):
  print(f'iteration: {i}')
  helper.generate_my_gen(args)
  helper.train()
 
% ls .logs/ -a
% ls .logs/*/ -a
#% cat .logs/20210423/20210423_1358/execution_log.txt

50/50 [==============================] - 104s 2s/step - loss: 0.4217 - accuracy: 0.0000e+00 - mean_io_u_10: 0.4815 - precision_10: 0.0298 - recall_10: 0.2642
Epoch 00004: early stopping
iteration: 5
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 640, 896, 3) 0                                            
__________________________________________________________________________________________________
conv2d_264 (Conv2D)             (None, 640, 896, 64) 1792        input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_265 (Conv2D)             (None, 640, 896, 64) 36928       conv2d_264[0][0]                 
__________________________________________________________________________________________

ResourceExhaustedError: ignored

##Ou criar .npy data para treinar
###If your computer has enough memory, you can create npy files containing all your images and masks, and feed your DNN with them.



In [ ]:
image_arr,mask_arr = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#np.save("data/image_arr.npy",image_arr)
#np.save("data/mask_arr.npy",mask_arr)

Train with npy file

In [ ]:
#imgs_train, imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

#Test e avaliação

Executar o test

In [ ]:
args.t = 1
args.n = "2021.04.27_010303_unet.hdf5"

helper.test(args)

Avaliação

In [ ]:
from google.colab import files
files.download(args.n)

In [ ]:
#from PIL import Image
#import numpy as np
#from matplotlib import pyplot as plt 


#model = helper.get_model()
#model.load_weights("train_weights/treino_001_unet.hdf5")
helper.generate_my_gen(args)
train_generator = helper.my_gene

x_test = []
y_test = []

# x1 = None, x2 = None, x3 = None, x4 = None 
i = 0

print(f'{i}/{len(list(train_generator))}')

for each in train_generator:
  i = i + 1
  if (i % 10 == 0):
      print(f'{i}/{len(train_generator.)}')
  x_test.append(each[0])
  y_test.append(each[1])
  # for j in range(0, helper.batch_size):
  #   x_test.append(each[0][j])
  #   y_test.append(each[1][j])

  # quatro, pq o batch size é 4
  #x1, x2, x3, x4 = each[0]
  #y1, y2, y3, y4 = each[1]
  
#x_test = np.array(x_test)
#y_test = np.array(y_test)
#print(x_test.shape)
#print(y_test.shape)

#helper.showSummary(args)
print(f'len - x_test:{len(x_test)} - y_test:{len(y_test)}')
#helper.evaluate(x_test, y_test)
if(not helper.model):
  helper.model = helper.get_model()

helper.model.evaluate(x=x_test, y=y_test)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

In [ ]:
plt.imshow(x1) # x1, x2, x3, x4 

In [ ]:
plt.imshow(y1) # y1, y2, y3, y4 

##Ou criar .npy data para treinar
###If your computer has enough memory, you can create npy files containing all your images and masks, and feed your DNN with them.



In [ ]:
image_arr,mask_arr = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#np.save("data/image_arr.npy",image_arr)
#np.save("data/mask_arr.npy",mask_arr)

Train with npy file

In [ ]:
#imgs_train, imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

# Load the TensorBoard notebook extension

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

montar gráficos

In [ ]:
! ls .logs/
%tensorboard --logdir .logs/

Test - Predição das imaegns